In [15]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import math
import pandas as pd
from scipy.optimize import curve_fit
import uncertainties
from uncertainties import ufloat
from uncertainties import unumpy

from tabulate import tabulate

In [16]:
import os

# Change to the project directory
os.chdir('/home/ursuni/Research_Project')

# Confirm the current working directory
print("Current working directory:", os.getcwd())

Current working directory: /home/ursuni/Research_Project


In [17]:
def get_param_1D(path):
    data = np.load(path, allow_pickle=True)
    
    popt_mbb_array = data['popt_mbb']
    pcov_mbb_array = data['pcov_mbb']
    popt_myy_array = data['popt_myy']
    pcov_myy_array = data['pcov_myy']

    mu_signal_mbb = ufloat(popt_mbb_array[0][0], np.sqrt(pcov_mbb_array[0][0][0]))
    mu_bkg_mbb = ufloat(popt_mbb_array[0][1], np.sqrt(pcov_mbb_array[0][1][1]))

    mu_signal_myy = ufloat(popt_myy_array[0][0], np.sqrt(pcov_myy_array[0][0][0]))
    mu_bkg_myy = ufloat(popt_myy_array[0][1], np.sqrt(pcov_myy_array[0][1][1]))
    
    data.close()

    return mu_signal_mbb, mu_bkg_mbb, mu_signal_myy, mu_bkg_myy

In [18]:
def get_param_2D(path):
    data = np.load(path, allow_pickle=True)
    
    popt = data['popt']
    pcov = data['pcov']

    mu_s = ufloat(popt[0], np.sqrt(pcov[0][0]))
    mu_b = ufloat(popt[1], np.sqrt(pcov[1][1]))
    
    data.close()

    return mu_s, mu_b

In [19]:
binning = np.array(['20-60', '20-19', '20-15', '20-9', '15-9', '11-9', '11-7'])

def get_1D(dataset):
    mu_s_mbb_list = []
    mu_b_mbb_list = []
    mu_s_myy_list = []
    mu_b_myy_list = []
    
    for entry in binning:
        mu_s_mbb, mu_b_mbb, mu_s_myy, mu_b_myy = \
            get_param_1D('HM-SampleArrays/HM-Y=3/Y=3-' + dataset + '-' + entry + '-Signal.npz')
        
        mu_s_mbb_list.append(mu_s_mbb)
        mu_b_mbb_list.append(mu_b_mbb)
        mu_s_myy_list.append(mu_s_myy)
        mu_b_myy_list.append(mu_b_myy)

    return mu_s_mbb_list, mu_b_mbb_list, mu_s_myy_list, mu_b_myy_list

In [6]:
def get_2D(dataset):
    mu_s_list = []
    mu_b_list = []
    
    for entry in binning:
        mu_s, mu_b = get_param_2D('HM-SampleArrays/HM-Y=3/Y=3-2D-' + \
                                  dataset + '-' + entry + '-Signal.npz')
        
        mu_s_list.append(mu_s)
        mu_b_list.append(mu_b)

    return mu_s_list, mu_b_list

In [7]:
mu_s_mbb_old_nom, mu_b_mbb_old_nom, mu_s_myy_old_nom, mu_b_myy_old_nom = \
get_1D('old_nominal')

mu_s_mbb_new_nom, mu_b_mbb_new_nom, mu_s_myy_new_nom, mu_b_myy_new_nom = \
get_1D('new_nominal')

mu_s_mbb_no_mbb, mu_b_mbb_no_mbb, mu_s_myy_no_mbb, mu_b_myy_no_mbb = \
get_1D('no_mbb')

In [8]:
mu_s_old_nom_2D, mu_b_old_nom_2D = get_2D('old_nominal')

mu_s_new_nom_2D, mu_b_new_nom_2D = get_2D('new_nominal')

mu_s_no_mbb_2D, mu_b_no_mbb_2D = get_2D('no_mbb')

In [21]:
old_nom_table = np.array([binning, mu_s_mbb_old_nom, \
                          mu_s_myy_old_nom, mu_s_old_nom_2D]).T

print('Table for mu_signal (old_nominal):\n')

print(tabulate(old_nom_table, tablefmt='latex', \
               headers=['configuration (mbb x myy)', '1D-mbb', '1D-myy', '2D']))

Table for mu_signal (old_nominal):

\begin{tabular}{llll}
\hline
 configuration (mbb x myy)   & 1D-mbb   & 1D-myy    & 2D        \\
\hline
 20-60                       & 0+/-15   & 0.9+/-1.9 & 1.0+/-3.3 \\
 20-19                       & 0+/-15   & 0.9+/-1.8 & 0.9+/-2.3 \\
 20-15                       & 0+/-15   & 0.9+/-1.8 & 1.0+/-2.2 \\
 20-9                        & 0+/-15   & 0.9+/-1.8 & 1.0+/-2.0 \\
 15-9                        & 0+/-15   & 0.9+/-1.8 & 1.0+/-1.9 \\
 11-9                        & 0+/-15   & 0.9+/-1.8 & 1.0+/-1.9 \\
 11-7                        & 0+/-15   & 0.9+/-1.8 & 0.9+/-1.8 \\
\hline
\end{tabular}


In [23]:
new_nom_table = np.array([binning, mu_s_mbb_new_nom, \
                          mu_s_myy_new_nom, mu_s_new_nom_2D]).T

print('Table for mu_signal (new_nominal):\n')

print(tabulate(new_nom_table, tablefmt='latex', \
               headers=['configuration (mbb x myy)', '1D-mbb', '1D-myy', '2D']))

Table for mu_signal (new_nominal):

\begin{tabular}{llll}
\hline
 configuration (mbb x myy)   & 1D-mbb   & 1D-myy    & 2D        \\
\hline
 20-60                       & 0+/-16   & 1.0+/-1.9 & 1.0+/-3.4 \\
 20-19                       & 0+/-16   & 1.0+/-1.8 & 1.0+/-2.3 \\
 20-15                       & 0+/-16   & 1.0+/-1.8 & 1.0+/-2.2 \\
 20-9                        & 0+/-16   & 1.0+/-1.8 & 1.0+/-2.0 \\
 15-9                        & 0+/-16   & 1.0+/-1.8 & 1.0+/-1.9 \\
 11-9                        & 0+/-16   & 1.0+/-1.8 & 1.0+/-1.9 \\
 11-7                        & 0+/-16   & 1.0+/-1.8 & 1.0+/-1.8 \\
\hline
\end{tabular}


In [24]:
no_mbb_table = np.array([binning, mu_s_mbb_no_mbb, \
                          mu_s_myy_no_mbb, mu_s_no_mbb_2D]).T

print('Table for mu_signal (no_mbb):\n')

print(tabulate(no_mbb_table, tablefmt='latex', \
               headers=['configuration (mbb x myy)', '1D-mbb', '1D-myy', '2D']))

Table for mu_signal (no_mbb):

\begin{tabular}{llll}
\hline
 configuration (mbb x myy)   & 1D-mbb   & 1D-myy    & 2D        \\
\hline
 20-60                       & 0+/-6    & 0.9+/-2.0 & 1+/-4     \\
 20-19                       & 0+/-6    & 0.9+/-1.9 & 0.9+/-2.4 \\
 20-15                       & 0+/-6    & 0.9+/-1.9 & 0.9+/-2.3 \\
 20-9                        & 0+/-6    & 0.9+/-1.9 & 0.9+/-2.0 \\
 15-9                        & 0+/-6    & 0.9+/-1.9 & 0.9+/-1.9 \\
 11-9                        & 0+/-6    & 0.9+/-1.9 & 0.9+/-1.9 \\
 11-7                        & 0+/-6    & 0.9+/-1.9 & 0.9+/-1.8 \\
\hline
\end{tabular}


In [25]:
old_nom_table_bkg = np.array([binning, mu_b_mbb_old_nom, \
                          mu_b_myy_old_nom, mu_b_old_nom_2D]).T

print('Table for mu_bkg (old_nominal):\n')

print(tabulate(old_nom_table_bkg, tablefmt='latex', \
               headers=['configuration (mbb x myy)', '1D-mbb', '1D-myy', '2D']))

Table for mu_bkg (old_nominal):

\begin{tabular}{llll}
\hline
 configuration (mbb x myy)   & 1D-mbb    & 1D-myy      & 2D          \\
\hline
 20-60                       & 1.1+/-1.0 & 1.0+/-0.4   & 1.0+/-0.8   \\
 20-19                       & 1.1+/-1.0 & 1.01+/-0.32 & 1.0+/-0.5   \\
 20-15                       & 1.1+/-1.0 & 1.01+/-0.32 & 1.0+/-0.5   \\
 20-9                        & 1.1+/-1.0 & 1.00+/-0.31 & 1.0+/-0.4   \\
 15-9                        & 1.1+/-1.0 & 1.00+/-0.31 & 1.0+/-0.4   \\
 11-9                        & 1.1+/-1.0 & 1.00+/-0.31 & 1.0+/-0.4   \\
 11-7                        & 1.1+/-1.0 & 1.01+/-0.31 & 1.00+/-0.35 \\
\hline
\end{tabular}


In [26]:
new_nom_table_bkg = np.array([binning, mu_b_mbb_new_nom, \
                          mu_b_myy_new_nom, mu_b_new_nom_2D]).T

print('Table for mu_bkg (new_nominal):\n')

print(tabulate(new_nom_table_bkg, tablefmt='latex', \
               headers=['configuration (mbb x myy)', '1D-mbb', '1D-myy', '2D']))

Table for mu_bkg (new_nominal):

\begin{tabular}{llll}
\hline
 configuration (mbb x myy)   & 1D-mbb    & 1D-myy      & 2D        \\
\hline
 20-60                       & 1.1+/-1.1 & 1.0+/-0.4   & 1.0+/-0.8 \\
 20-19                       & 1.1+/-1.1 & 1.00+/-0.33 & 1.0+/-0.5 \\
 20-15                       & 1.1+/-1.1 & 1.00+/-0.33 & 1.0+/-0.5 \\
 20-9                        & 1.1+/-1.1 & 1.00+/-0.33 & 1.0+/-0.4 \\
 15-9                        & 1.1+/-1.1 & 1.00+/-0.33 & 1.0+/-0.4 \\
 11-9                        & 1.1+/-1.1 & 1.00+/-0.33 & 1.0+/-0.4 \\
 11-7                        & 1.1+/-1.1 & 1.00+/-0.32 & 1.0+/-0.4 \\
\hline
\end{tabular}


In [27]:
no_mbb_table_bkg = np.array([binning, mu_b_mbb_no_mbb, \
                          mu_b_myy_no_mbb, mu_b_no_mbb_2D]).T

print('Table for mu_bkg (no_mbb):\n')

print(tabulate(no_mbb_table_bkg, tablefmt='latex',  \
               headers=['configuration (mbb x myy)', '1D-mbb', '1D-myy', '2D']))

Table for mu_bkg (no_mbb):

\begin{tabular}{llll}
\hline
 configuration (mbb x myy)   & 1D-mbb    & 1D-myy      & 2D        \\
\hline
 20-60                       & 1.1+/-0.5 & 1.0+/-0.4   & 1.0+/-1.1 \\
 20-19                       & 1.1+/-0.5 & 1.01+/-0.33 & 1.0+/-0.6 \\
 20-15                       & 1.1+/-0.5 & 1.01+/-0.33 & 1.0+/-0.6 \\
 20-9                        & 1.1+/-0.5 & 1.01+/-0.32 & 1.0+/-0.5 \\
 15-9                        & 1.1+/-0.5 & 1.01+/-0.32 & 1.0+/-0.5 \\
 11-9                        & 1.1+/-0.5 & 1.01+/-0.32 & 1.0+/-0.4 \\
 11-7                        & 1.1+/-0.5 & 1.01+/-0.32 & 1.0+/-0.4 \\
\hline
\end{tabular}
